In [168]:
from functools import partial
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import polars as pl
from sqlalchemy.sql.functions import coalesce

import Helper_Functions_Portfolio as hf
from cmds.portfolio_management_helper import calc_summary_statistics

import hvplot.polars


import warnings
warnings.filterwarnings("ignore")

In [169]:
import importlib
importlib.reload(hf)

<module 'Helper_Functions_Portfolio' from 'C:\\Users\\Gurjivan\\DataspellProjects\\finm-portfolio-2024\\GSK_HW\\Helper_Functions_Portfolio.py'>

In [170]:
signals = pl.read_excel("..\data\gmo_data.xlsx",sheet_name = "signals")
rf = pl.read_excel("..\data\gmo_data.xlsx",sheet_name = "risk-free rate")
total_returns = pl.read_excel("..\data\gmo_data.xlsx",sheet_name = "total returns")

In [171]:
excess_returns = total_returns.join(rf,on = "date",how = "inner").with_columns(
    (total_returns["GMWAX"] - (rf["TBill 3M"])/12).alias("GMWAX"),
    (total_returns["SPY"] - (rf["TBill 3M"])/12).alias("SPY"),
    (total_returns["GMGEX"] - (rf["TBill 3M"])/12).alias("GMGEX")
)

Number 1

This section utilizes data in the file, gmo
 data.xlsx.
 Examine GMO’s performance. Use the risk-free rate to convert the total returns to excess returns
 1. Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:
 • from inception through 2011
 • 2012-present
 • inception- present

 Has the mean, vol, and Sharpe changed much since the case?

In [172]:
print("Before 2012")
GMWAX_excess_2012=excess_returns.select("date","GMWAX").filter((pl.col("date").dt.year())<2012)
GMWAX_excess_greater2012=excess_returns.select("date","GMWAX").filter((pl.col("date").dt.year())>2012)
GMWAX_excess = excess_returns.select("date","GMWAX")
hf.Portfolio_Statistics(GMWAX_excess_2012,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")

Before 2012


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMWAX""",0.046422,0.110499,0.42011


In [173]:
print("After 2012")
hf.Portfolio_Statistics(GMWAX_excess_greater2012,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")


After 2012


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMWAX""",0.038547,0.096626,0.398926


In [174]:
print("All")
hf.Portfolio_Statistics(GMWAX_excess,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")


All


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMWAX""",0.045043,0.10349,0.43524


The mean has actually gone done since the beginning of the case as shown in the period 2012 to present. The volatility has also gone down since the case. The Sharpe ratio has also gone down since the case. But the sharpe ratio has also gone down since then

2.  believes a risk premium is compensation for a security’s tendency to lose money at “bad
 times”. For all three samples, analyze extreme scenarios by looking at
 • Min return
 • 5th percentile (VaR-5th)
 • Maximum drawdown1
 (a) Does GMWAX have high or low tail-risk as seen by these stats?
 (b) Does that vary much across the two subsamples?

In [175]:
print("Before 2012")
hf.Drawdown_Statistics(GMWAX_excess_2012).select("Stock","Max_Drawdown").join(
    hf.Portfolio_Statistics(GMWAX_excess_2012,annual_factor = 12,quantile=0.05).select(
        "Stock","VaR","min"),on = "Stock")

Before 2012


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMWAX""",-0.30652,-0.044003,-0.14915


In [176]:
print("After 2012")
hf.Drawdown_Statistics(GMWAX_excess_greater2012).select("Stock", "Max_Drawdown").join(
    hf.Portfolio_Statistics(GMWAX_excess_greater2012, annual_factor = 12, quantile = 0.05).select(
        "Stock", "VaR", "min"), on = "Stock")

After 2012


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMWAX""",-0.225613,-0.040517,-0.115018


In [177]:
print("All")
hf.Drawdown_Statistics(GMWAX_excess).select("Stock", "Max_Drawdown").join(
    hf.Portfolio_Statistics(GMWAX_excess, annual_factor = 12, quantile = 0.05).select(
        "Stock", "VaR", "min"), on = "Stock")

All


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMWAX""",-0.30652,-0.041368,-0.14915


(a) Does GMWAX have high or low tail-risk as seen by these stats?

It seems the GMWAX has low tail risk as seen by the stats. The reasons for this include lower expected growth rates and higher current valuations 

 (b) Does that vary much across the two subsamples?
 
Yes, expect the max drawdown, VaR are lower are lower in the earlier subperiod compared to the other periods.

 3. For all three samples, regress excess returns of GMWAX on excess returns of SPY.
 (a) Report the estimated alpha, beta, and r-squared.
 (b) Is GMWAX a low-beta strategy? Has that changed since the case?
 (c) Does GMWAX provide alpha? Has that changed across the subsamples?

In [178]:
SPY_excess = excess_returns.select("date","SPY")
SPY_excess_2012=excess_returns.select("date","SPY").filter((pl.col("date").dt.year())<2012)
SPY_excess_greater2012=excess_returns.select("date","SPY").filter((pl.col("date").dt.year())>2012)

In [179]:
print("All")
result = hf.Regression_Stats(GMWAX_excess.select("GMWAX"), SPY_excess.select("SPY"), intercept=True, annual_factor=12)
filtered_result = result.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result

All


Metric,Value
str,f64
"""Beta_SPY""",0.552608
"""R-Squared""",0.680167
"""Annualized Alpha""",0.000558


In [180]:
print("After 2012")
result_2 = hf.Regression_Stats(GMWAX_excess_greater2012.select("GMWAX"), SPY_excess_greater2012.select("SPY"), intercept=True, annual_factor=12)
filtered_result_2 = result_2.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result_2

After 2012


Metric,Value
str,f64
"""Beta_SPY""",0.578472
"""R-Squared""",0.74425
"""Annualized Alpha""",-0.037384


In [181]:
print("Before 2012")
result_3 = hf.Regression_Stats(GMWAX_excess_2012.select("GMWAX"), SPY_excess_2012.select("SPY"), intercept=True, annual_factor=12)
filtered_result_3 = result_3.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result_3


Before 2012


Metric,Value
str,f64
"""Beta_SPY""",0.542128
"""R-Squared""",0.648686
"""Annualized Alpha""",0.027


(b) Is GMWAX a low-beta strategy? Has that changed since the case?

GMWAX is a low beta strategy as the beta is less than 1. The beta has actually gone down since the case.

 (c) Does GMWAX provide alpha? Has that changed across the subsamples?
 
GMWAX does not provide alpha as the alpha is negative. The alpha before the case were positive but after the case they have been negative.

 4. Above, we’ve evaluated GMO’s macro fund, ”GMWAX”, as studied in the case. Now, consider
 GMO’s equity fund, ”GMGEX”.
 Compute the performance stats of 3.1-3.3 for GMGEX. What are some of the major differences
 between these two strategies?

In [182]:
print("Before 2012")
GMGEX_excess_2012=excess_returns.select("date","GMGEX").filter((pl.col("date").dt.year())<2012)
GMGEX_excess_greater2012=excess_returns.select("date","GMGEX").filter((pl.col("date").dt.year())>2012)
GMGEX_excess = excess_returns.select("date","GMGEX")
hf.Portfolio_Statistics(GMGEX_excess_2012,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")

Before 2012


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMGEX""",-0.003823,0.147253,-0.025963


In [183]:
print("After 2012")
hf.Portfolio_Statistics(GMGEX_excess_greater2012,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")


After 2012


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMGEX""",-0.006115,0.242852,-0.025178


In [184]:
print("All")
hf.Portfolio_Statistics(GMGEX_excess,annual_factor = 12,quantile=0.05).select("Stock","mean","std","sharpe")


All


Stock,mean,std,sharpe
str,f64,f64,f64
"""GMGEX""",-0.001463,0.192622,-0.007595


In [185]:
print("Before 2012")
hf.Drawdown_Statistics(GMGEX_excess_2012).select("Stock","Max_Drawdown").join(
    hf.Portfolio_Statistics(GMGEX_excess_2012,annual_factor = 12,quantile=0.05).select(
        "Stock","VaR","min"),on = "Stock")

Before 2012


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMGEX""",-0.563989,-0.082292,-0.151592


In [186]:
print("All")
hf.Drawdown_Statistics(GMGEX_excess).select("Stock","Max_Drawdown").join(
    hf.Portfolio_Statistics(GMGEX_excess,annual_factor = 12,quantile=0.05).select(
        "Stock","VaR","min"),on = "Stock")

All


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMGEX""",-0.768086,-0.076213,-0.658863


In [187]:
print("After 2012")
hf.Drawdown_Statistics(GMGEX_excess_greater2012).select("Stock","Max_Drawdown").join(
    hf.Portfolio_Statistics(GMGEX_excess_greater2012,annual_factor = 12,quantile=0.05).select(
        "Stock","VaR","min"),on = "Stock")


After 2012


Stock,Max_Drawdown,VaR,min
str,f64,f64,f64
"""GMGEX""",-0.738304,-0.066255,-0.658863


In [188]:
print("All")
result_4 = hf.Regression_Stats(GMGEX_excess.select("GMGEX"), SPY_excess.select("SPY"), intercept=True, annual_factor=12)
filtered_result_4 = result_4.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result_4

All


Metric,Value
str,f64
"""Beta_SPY""",0.786683
"""R-Squared""",0.397891
"""Annualized Alpha""",-0.06479


In [189]:
print("Before 2012")
result_5 = hf.Regression_Stats(GMGEX_excess_2012.select("GMGEX"), SPY_excess_2012.select("SPY"), intercept=True, annual_factor=12)
filtered_result_5 = result_5.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result_5

Before 2012


Metric,Value
str,f64
"""Beta_SPY""",0.764237
"""R-Squared""",0.725898
"""Annualized Alpha""",-0.031201


In [190]:
print("After 2012")
result_6 = hf.Regression_Stats(GMGEX_excess_greater2012.select("GMGEX"), SPY_excess_greater2012.select("SPY"), intercept=True, annual_factor=12)
filtered_result_6 = result_6.filter(pl.col("Metric").is_in(["Annualized Alpha", "Beta_SPY", "R-Squared"]))
filtered_result_6

After 2012


Metric,Value
str,f64
"""Beta_SPY""",0.827185
"""R-Squared""",0.240914
"""Annualized Alpha""",-0.114691


Compute the performance stats of 3.1-3.3 for GMGEX. What are some of the major differences
 between these two strategies?
 
GMGEX first of all appears to have negative mean returns no matter which period showing that it has underporfomred GWMAX. In addition to this it takes on alot more risk as the drawdown were much greater almost 70% compared to around 40% for GMWAX. The VaR was also much higher for GMGEX. The beta for GMGEX was also much higher than GMWAX. The alpha was also negative for GMGEX.

3- This section utilizes data in the file, gmo
 data.xlsx.
 1. Consider the lagged regression, where the regressor, (X,) is a period behind the target, (rSPY).
 rSPY
 t
 =αSPY,X +(βSPY,X)′Xt−1 +ϵSPY,X
 t
 Estimate (1) and report the R2, as well as the OLS estimates for α and β. Do this for...
 1This should be calculated on GMWAX total returns, not excess returns.
 (1)
 2
• X as a single regressor, the dividend-price ratio.
 • X as a single regressor, the earnings-price ratio.
 • X as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10-year
 yield.
 For each, report the r-squared.


 2. For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.
 • Build the forecasted SPY returns: ˆ rSPY
 t+1 . Note that this denotes the forecast made using
 Xt to forecast the (t +1) return.
 • Set the scale of the investment in SPY equal to 100 times the forecasted value:
 wt =100 ˆ rSPY
 t+1
 We are not taking this scaling too seriously. We just want the strategy to go bigger in
 periods where the forecast is high and to withdraw in periods where the forecast is low, or
 even negative.
 • Calculate the return on this strategy:
 rx
 t+1 = wtrSPY
 t+1
 You should now have the trading strategy returns, rx for each of the forecasts. For each strategy,
 estimate
 • mean, volatility, Sharpe,
 • max-drawdown
 • market alpha
 • market beta
 • market Information ratio


 3. GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad
 times”. Let’s consider risk characteristics.
 (a) For both strategies, the market, and GMO, calculate the monthly VaR for π = .05. Just
 use the quantile of the historic data for this VaR calculation.
 (b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011.
 Does the dynamic portfolio above under-perform the risk-free rate over this time?
 (c) Based on the regression estimates, in how many periods do we estimate a negative risk
 premium?
 (d) Do you believe the dynamic strategy takes on extra risk??

3.1

In [191]:
gmo_total_ret = pd.read_excel('../data/gmo_data.xlsx',sheet_name = 'total returns', index_col = 0)
gmo_total_ret.index.name = 'Date'

In [192]:
path = "../data/gmo_data.xlsx"
rf = pd.read_excel(path,sheet_name = 'risk-free rate', index_col = 0)
rf.index.name = 'Date'

In [193]:
path = "../data/gmo_data.xlsx"
gmo_signals = pd.read_excel(path,sheet_name = 'signals', index_col = 0)
gmo_signals.index.name = 'Date'

In [194]:
gmo_excess_ret = gmo_total_ret.copy()
for col in gmo_excess_ret.columns:
    gmo_excess_ret[col] = gmo_excess_ret[col] - rf['TBill 3M']

gmo_excess_ret.tail()

,SPY,GMWAX,GMGEX
Date,,,
2024-06-28,-0.0183,-0.0607,-0.0666
2024-07-31,-0.0407,-0.0224,-0.0182
2024-08-30,-0.0278,-0.0364,-0.0316
2024-09-30,-0.0252,-0.0337,-0.0308
2024-10-31,-0.0543,-0.0823,-0.0849


In [195]:
def time_series_regression(portfolio, factors, multiple_factors = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
            ff_report.loc[portf, factors.columns[2]+ ' beta'] = res.params[3]
        else:
            ff_report.loc[portf, factors.name + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

In [196]:
SPY = gmo_total_ret.loc[:,['SPY']]
signal_1 = ['SPX DVD YLD']
factor_1 = gmo_signals.loc[:,signal_1].shift(1).squeeze()
factor_1 = pd.Series(factor_1)
signal_reg_1 = time_series_regression(SPY, factor_1, multiple_factors=False, resid=False)
signal_reg_1.index = ['SPX DVD YLD']
signal_reg_1


signal_2 = ['SPX P/E']
factor_2 = gmo_signals.loc[:,signal_2].shift(1).squeeze()
signal_reg_2 = time_series_regression(SPY, factor_2, multiple_factors=False, resid=False)
signal_reg_2.index = ['SPX P/E']
signal_reg_2


signal_3 = ['SPX DVD YLD','SPX P/E','TNote 10YR']
factor_3 = gmo_signals.loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']].shift(1)
signal_reg_3 = time_series_regression(SPY, factor_3, multiple_factors=True, resid=False)
signal_reg_3.index = ['SPX DVD YLD,SPX P/E,TNote 10YR']

display(signal_reg_1,signal_reg_2,signal_reg_3)

,alpha_hat,SPX DVD YLD beta,info_ratio,treynor_ratio,R-squared,Tracking Error
SPX DVD YLD,-0.1321,0.0108,-0.8589,9.4503,0.0094,0.1538


,alpha_hat,SPX P/E beta,info_ratio,treynor_ratio,R-squared,Tracking Error
SPX P/E,0.2564,-0.0006,1.6627,-158.0849,0.0039,0.1542


,alpha_hat,SPX DVD YLD beta,SPX P/E beta,TNote 10YR beta,info_ratio,treynor_ratio,R-squared,Tracking Error
"SPX DVD YLD,SPX P/E,TNote 10YR",-0.1850,0.0118,0.0003,-0.0007,-1.2034,8.6703,0.0102,0.1537


### 3.2) For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.
- Build the forecasted SPY returns: $\hat{r}^{SPY}_{t+1}$. Note that this denotes the forecast made using $X_t$ to forecast the $(t+1)$ return.
- Set the scale of the investment in SPY equal to 100 times the forecasted value:
$
w_t = 100 \hat{r}^{SPY}_{t+1}
$
- We are not taking this scaling too seriously. We just want the  strategy  to  go  bigger  inperiods where the forecast is high and to withdraw in periods where the forecast is low, or even negative.
- Calcualte the return on this strategy:
$
r^X_{t+1} = w_tr^{SPY}_{t+1}
$

#### You should now have the trading strategy returns, $r^x$ for each of the forecasts. For each strategy, estimate:
- mean, volatility, Sharpe,
- max-drawdown
- market alpha
- market beta
- market Information

In [197]:
DP_return = (gmo_signals.loc[:,'SPX DVD YLD'].shift(1).to_frame() * signal_reg_1['SPX DVD YLD beta'])+signal_reg_1['alpha_hat']/12
DP_return = DP_return.rename(columns={'SPX DVD YLD':'Forecasted Return'}) * 100
DP_forecast_return = pd.DataFrame(DP_return['Forecasted Return']*gmo_total_ret.loc[:,['SPY']]['SPY'], columns=DP_return.columns, index=DP_return.index)


EP_return = (gmo_signals.loc[:,'SPX P/E'].shift(1).to_frame() * signal_reg_2['SPX P/E beta'])+signal_reg_2['alpha_hat']/12
EP_return = EP_return.rename(columns={'SPX P/E':'Forecasted Return'}) * 100
EP_forecast_return = pd.DataFrame(EP_return['Forecasted Return']*gmo_total_ret.loc[:,['SPY']]['SPY'], columns=EP_return.columns, index=EP_return.index)


forecasted_rets = (np.array(gmo_signals.shift(1).loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']]) @ np.array(signal_reg_3.loc[:,['SPX DVD YLD beta','SPX P/E beta','TNote 10YR beta']].T))
multiple_factor_return = (pd.DataFrame(forecasted_rets,columns = ['Forecasted Return'],index= gmo_signals.index)) 
multiple_factor_return['Forecasted Return'] = (multiple_factor_return['Forecasted Return'] + float(signal_reg_3['alpha_hat']/12))*100
multiple_forecast_return = pd.DataFrame(multiple_factor_return['Forecasted Return'] *gmo_total_ret.loc[:,['SPY']]['SPY'], columns=multiple_factor_return.columns, index=multiple_factor_return.index)

In [198]:
def performance_summary(return_data):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*12)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [199]:
strategy = {'SPX DVD YLD': DP_forecast_return.dropna(),
          'SPX P/E': EP_forecast_return.dropna(),
          'SPX P/E-SPX DVD YLD-US10Y': multiple_forecast_return.dropna()
         }
factor = gmo_excess_ret.loc[:,['SPY']]
total_strategy_summary = []

for key,value in strategy.items():
    strat = strategy[key]
    strat_summary = performance_summary(strat)
    strat_summary.index = [key]
    strat_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['TBill 3M'] <0])
    strat_summary['Total Months'] = len(strat)
    ts = time_series_regression(strat, factor[strat.index[0]:].squeeze(), False)
    strat_summary['Market Beta'] = ts['SPY beta'].values
    strat_summary['Market Alpha'] = ts['alpha_hat'].values
    strat_summary['Market Information Ratio'] = ts['info_ratio'].values
    
    total_strategy_summary.append(strat_summary)
    
total_strategy_df = pd.concat(total_strategy_summary)
  
total_strategy_df.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
SPX DVD YLD,0.1112,0.1728,0.6435,-0.7163,0.7966,0.2342,2.1843
SPX P/E,0.0980,0.1438,0.6819,-0.5817,0.7300,0.2108,2.9070
SPX P/E-SPX DVD YLD-US10Y,0.1130,0.1736,0.6510,-0.6994,0.8040,0.2372,2.2179


### 3.3) GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". Let's consider risk characteristics.
### 3.3.a) For both strategies, the market, and GMO, calculate the monthly VaR for $\pi=.05$. Just use the quantile of the historic data for this VaR calculation.


In [200]:
market_summary = performance_summary(gmo_excess_ret.loc[:,['SPY']])
gmo_summary = performance_summary(gmo_excess_ret.loc[:,['GMWAX']].dropna())
strat_var= pd.concat([total_strategy_df.loc[:,['VaR (0.05)']],market_summary.loc[:,['VaR (0.05)']],gmo_summary.loc[:,['VaR (0.05)']]])
strat_var

,VaR (0.05)
SPX DVD YLD,-0.0594
SPX P/E,-0.0651
SPX P/E-SPX DVD YLD-US10Y,-0.0620
SPY,-0.0995
GMWAX,-0.0785


### 3.3.b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

All of the dynamic portfolios outperform the risk free rate during this time

In [201]:
strats = {'SPX DVD YLD': DP_forecast_return.dropna(),
          'SPX P/E': EP_forecast_return.dropna(),
          'SPX DVD YLD-SPX P/E-US10Y': multiple_forecast_return.dropna()
         }
strat_summary_0011 =[]
for k,v in strats.items():
    strat = (strats[k]['2000':'2011']['Forecasted Return']).to_frame('Forecasted Returns')
    perf_summary = performance_summary(strat)
    perf_summary.index = [k]
    strat_summary_0011.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary_0011)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown
SPX DVD YLD,0.0554,0.2187,0.2534,-0.7163
SPX P/E,0.0444,0.1681,0.2642,-0.5817
SPX DVD YLD-SPX P/E-US10Y,0.0521,0.2128,0.2449,-0.6994


### 3.3.c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?


In [202]:
neg_risk_premium = total_strategy_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
SPX DVD YLD,209,334,62.5749
SPX P/E,201,334,60.1796
SPX P/E-SPX DVD YLD-US10Y,216,334,64.6707


### 3.3.d) Do you believe the dynamic strategy takes on extra risk?

I believe that the dynamic strategy does not take on extra risk. The metrics from the risk tail stats do not suggest anything about the strategy taking on more risk than SPY. The VAR for SPY appears much lower (more negative so really greater) so it cannot be suggestive  of the strategy taking on more risk. In addition the max drawdowns are also more negative for spy.  


In [203]:
total_strategy_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
SPX DVD YLD,0.1112,0.1728,0.6435,-0.0594,-0.7163,0.7966,0.2342,2.1843
SPX P/E,0.0980,0.1438,0.6819,-0.0651,-0.5817,0.7300,0.2108,2.9070
SPX P/E-SPX DVD YLD-US10Y,0.1130,0.1736,0.6510,-0.0620,-0.6994,0.8040,0.2372,2.2179


In [204]:
market_summary.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown']]


,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown
SPY,-0.1567,0.1702,-0.9207,-0.0995,-0.9929


4 Out-of-Sample Forecasting
 This section utilizes data in the file, gmo
 data.xlsx.
 Reconsider the problem above, of estimating (1) for x. The reported R2 was the in-sample R2–it
 examined how well the forecasts fit in the sample from which the parameters were estimated.
 In particular, focus on the case of using both dividend-price and earnings-price as signals.
 Let’s consider the out-of-sample r-squared. To do so, we need to do the following:
 • Start at t = 60.
 • Estimate (1) only using data through time t.
 • Use the estimated parameters of (1), along with xt+1 to calculate the out-of-sample forecast for
 the following period, t + 1.
 ˆ
 rSPY
 t+1 = ˆ αSPY,x
 • Calculate the t +1 forecast error,
 t
 eforecast
 t+1
 +(βSPY,x)′ xt
 =rSPY
 t+1 − ˆ rSPY
 t+1
 • Move to t =61, and loop through the rest of the sample.
 You now have the time-series of out-of-sample prediction errors, ex.
 Calculate the time-series of out-of-sample prediction errors enull, which are based on the null forecast

In [205]:
def OOS_r2(df, factors, start):
    y = df['SPY']
    X = sm.add_constant(factors)

    forecast_err, null_err = [], []

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return ((1 - RSS/TSS),reg)

In [206]:

factor = gmo_signals.loc[:,'SPX P/E'].shift(1).to_frame()
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_ep = OOS_r2(fund_ret,factor,60)
OOS_RSquared_ep = reg_ep[0]
OOS_r2_ep = pd.DataFrame([[OOS_RSquared_ep]], columns = ['OOS R-Squared'], index = ['SPX P/E'])
reg_ep_params = reg_ep[1]

In [207]:
factor = gmo_signals.loc[:,'SPX DVD YLD'].shift(1).to_frame()
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_dp = OOS_r2(fund_ret,factor,60)
OOS_RSquared_dp = reg_dp[0]
OOS_r2_dp = pd.DataFrame([[OOS_RSquared_dp]], columns = ['OOS R-Squared'], index = ['SPX DVD YLD'])
reg_dp_params = reg_dp[1]

In [208]:
factor = gmo_signals.loc[:,['SPX DVD YLD','SPX P/E']].shift(1)
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_epdp = OOS_r2(fund_ret,factor,60)
OOS_r2_epdp  = reg_epdp[0]
OOS_r2_epdp = pd.DataFrame([[OOS_r2_epdp]], columns = ['OOS R-Squared'], index = ['SPX DVD YLD-SPX P/E'])
reg_epdp_params = reg_epdp[1]

In [209]:
factor = gmo_signals.loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']].shift(1)
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_all = OOS_r2(fund_ret,factor,60)
OOS_RSquared_all  = reg_all[0]
OOS_r2_all = pd.DataFrame([[OOS_RSquared_all]], columns = ['OOS R-Squared'], index = ['All'])
reg_all_params = reg_all[1]

### 4.1) Report the out-of-sample $R^2$:
\begin{align}
 R^2_{OOS} \equiv 1-\frac{\sum^T_{i=61}(e^x_i)^2}{\sum^T_{i=61}(e^0_i)^2} 
\end{align}
### note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between $(-\infty,1]$.

In [210]:
oos_r2_sum = pd.concat([OOS_r2_dp,OOS_r2_ep,OOS_r2_epdp,OOS_r2_all])
oos_r2_sum

,OOS R-Squared
SPX DVD YLD,-0.0456
SPX P/E,-0.0138
SPX DVD YLD-SPX P/E,-0.0643
All,-0.0809


 Did this forecasting strategy produce a positive OOS r-squared?
 
No the strategy did not produce a postive r-squared. They were all negative. 

### 4.2) Re-do problem 3.2 using this OOS forecast. <br><br> How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2?

The Out-of-Sample performs significantly worse than in-sample as expected. With lower mean, much higher volatility and tail risk measure, the oos performance seems to be taking much higher risk with a large negative beta to the market returns.

In [211]:
def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(factors)

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [212]:
factor = gmo_signals.loc[:,'SPX P/E'].shift(1).to_frame()
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_EP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'SPX_P/E_OOS_Returns'})

In [213]:
factor = gmo_signals.loc[:,'SPX DVD YLD'].shift(1).to_frame()
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_DP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'SPX DVD YLD_OOS_Returns'})

In [214]:
factor = gmo_signals.loc[:,['SPX DVD YLD','SPX P/E']].shift(1)
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_EPDP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'SPX DVD YLD-SPX P/E_OOS_Returns'})

In [215]:
factor = gmo_signals.loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']].shift(1)
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_all_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'All_OOS_Returns'})

In [216]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T, OOS_EP_predict.T, OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'SPX DVD YLD': OOS_DP_predict.dropna(),
          'SPX_P/E': OOS_EP_predict.dropna(),
          'SPX DVD YLD-SPX P/E': OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'SPY': gmo_excess_ret.loc[OOS_all_predict.index[0]:, ['SPY']].rename(columns = {'SPY': 'SPY_OOS_Returns'}),
          'TBill 3M': rf['TBill 3M'].to_frame('TBill 3M_OOS_Returns')
          }
factor = gmo_excess_ret.loc[:, ['SPY']]
strat_summary = []
for k, v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k + '_OOS_Returns'] - rf['TBill 3M'] < 0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze(), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,
['Mean', 'Volatility', 'Sharpe Ratio', 'VaR (0.05)', 'Max Drawdown', 'Market Beta', 'Market Alpha',
 'Market Information Ratio']]
### 4.3) Re-do problem 3.3 using this OOS forecast. <br><br> Is the point-in-time version of the strategy riskier?

### 4.3) Re-do problem 3.3 using this OOS forecast. <br><br> Is the point-in-time version of the strategy riskier?

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
SPX DVD YLD,0.0815,0.1945,0.4189,-0.0613,-0.6854,0.7683,0.1481,0.9873
SPX_P/E,0.1013,0.1574,0.6439,-0.0628,-0.5415,0.7615,0.1674,1.6978
SPX DVD YLD-SPX P/E,0.0760,0.2345,0.3240,-0.0668,-0.7425,0.5993,0.1280,0.5988
All,0.0396,0.2153,0.1841,-0.0620,-0.7099,0.4015,0.0745,0.3626
SPY,-0.0867,0.1611,-0.5384,-0.0929,-0.9163,1.0000,-0.0000,-0.9335
TBill 3M,0.2588,0.0728,3.5529,0.0001,-0.0002,-0.1802,0.2305,3.4896


How much better/worse is the OOS strategy compared to the in-sample version of 3.2?

This strategy performed much worse out of sample which is expected since the predicitve power isnt going to be as great. This is shown through the lower mean returns and sharpes

### 4.3) Re-do problem 3.3 using this OOS forecast. <br><br> Is the point-in-time version of the strategy riskier?
The mean returns appear to go down significatly in this period compared to the ful sub sample. In addition the volatility and increases while the sharep ratio also decreases. So based on this the strategy does take on more risk but the mean returns for this period suggest the risk may not have been worth it. 

In [217]:
os_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'SPX DVD YLD': OOS_DP_predict.dropna(),
          'SPX_P/E': OOS_EP_predict.dropna(),
          'SPX DVD YLD-SPX P/E':OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'TBill 3M':rf['TBill 3M'].to_frame('TBill 3M_OOS_Returns')
         }
factor = gmo_excess_ret.loc[:,['SPY']]['2000':'2011']
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]['2000':'2011']
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['TBill 3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze(), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
SPX DVD YLD,0.0322,0.2661,0.1209,-0.1286,-0.6854,0.9258,0.1951,0.9025
SPX_P/E,0.1006,0.2044,0.4921,-0.0703,-0.5415,0.9729,0.2718,2.2029
SPX DVD YLD-SPX P/E,0.0275,0.3341,0.0822,-0.1532,-0.7425,0.6126,0.1353,0.4254
All,-0.0227,0.3058,-0.0744,-0.1349,-0.7098,0.2202,0.0160,0.0527
TBill 3M,0.2745,0.0694,3.9564,0.0004,-0.0002,-0.1655,0.2321,3.7003


In [218]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
SPX DVD YLD,147,275,53.4545
SPX_P/E,152,275,55.2727
SPX DVD YLD-SPX P/E,150,275,54.5455
All,155,275,56.3636
SPY,179,275,65.0909
TBill 3M,0,335,0.0000
